In [ ]:
import requests
import csv
import time
import logging
import os
import signal
import sys
from datetime import datetime
from collections import deque

# Configuration
API_KEY = "TO-BE-UPDATED-TO-YOUR-API-KEY"
REGION = "euw1"
REGIONAL_CODE = "europe"
TIER = "DIAMOND"
DIVISION = "I"
LANE = "TOP"
MAX_PLAYERS = 100  # Number of players to process per batch
MATCHES_PER_PLAYER = 20  # Matches to check per player

# Logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='matchup_data.log'
)

# Rate limiter class for API constraints
class RateLimiter:
    def __init__(self):
        self.short_window = deque()  # 1-second window
        self.long_window = deque()   # 2-minute window

    def wait(self):
        """Enforce rate limits: 20 requests/sec and 100 requests/2min"""
        while True:
            now = time.time()
            # Clean expired requests
            self._clean_window(self.short_window, 1)
            self._clean_window(self.long_window, 120)

            # Calculate required wait time
            sleep_time = max(
                self._calc_wait_time(self.short_window, 20, 1),
                self._calc_wait_time(self.long_window, 100, 120)
            )
            
            if sleep_time > 0:
                time.sleep(sleep_time + 0.1)  # Add buffer
            else:
                break

        # Record new request
        timestamp = time.time()
        self.short_window.append(timestamp)
        self.long_window.append(timestamp)

    def _clean_window(self, window, interval):
        """Remove outdated requests from the window"""
        now = time.time()
        while window and window[0] < now - interval:
            window.popleft()

    def _calc_wait_time(self, window, limit, interval):
        """Calculate required wait time for a window"""
        if len(window) >= limit:
            return interval - (time.time() - window[0])
        return 0

# Initialize rate limiter
limiter = RateLimiter()

def get_players():
    """Fetch current tier players from Riot API"""
    try:
        limiter.wait()
        response = requests.get(
            f"https://{REGION}.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{TIER}/{DIVISION}",
            headers={"X-Riot-Token": API_KEY}
        )
        response.raise_for_status()
        return response.json()[:MAX_PLAYERS]
    except Exception as e:
        logging.error(f"Failed to get players: {str(e)}")
        return []

def get_puuid(summoner_id):
    """Get PUUID for a summoner"""
    try:
        limiter.wait()
        response = requests.get(
            f"https://{REGION}.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}",
            headers={"X-Riot-Token": API_KEY}
        )
        response.raise_for_status()
        return response.json().get("puuid")
    except Exception as e:
        logging.error(f"Failed to get PUUID: {str(e)}")
        return None

def get_match_history(puuid):
    """Retrieve match history for a player"""
    try:
        limiter.wait()
        response = requests.get(
            f"https://{REGIONAL_CODE}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids",
            params={"queue": 420, "count": MATCHES_PER_PLAYER},
            headers={"X-Riot-Token": API_KEY}
        )
        response.raise_for_status()
        return response.json()
    except Exception as e:
        logging.error(f"Failed to get match history: {str(e)}")
        return []

def process_match(match_id, writer):
    """Process a single match and write data to CSV"""
    try:
        # Get match data
        limiter.wait()
        match_data = requests.get(
            f"https://{REGIONAL_CODE}.api.riotgames.com/lol/match/v5/matches/{match_id}",
            headers={"X-Riot-Token": API_KEY}
        ).json()

        # Get timeline data
        limiter.wait()
        timeline_data = requests.get(
            f"https://{REGIONAL_CODE}.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline",
            headers={"X-Riot-Token": API_KEY}
        ).json()

        participants = match_data["info"]["participants"]
        game_duration = match_data["info"]["gameDuration"] / 60  # Convert to minutes

        records = []
        for player in [p for p in participants if p["teamPosition"] == LANE]:
            try:
                # Find lane opponent
                opponent = next(
                    p for p in participants 
                    if p["teamPosition"] == LANE and p["teamId"] != player["teamId"]
                )
            except StopIteration:
                continue

            # Build data record
            record = {
                "match_id": match_id,
                "timestamp": datetime.fromtimestamp(match_data["info"]["gameCreation"]/1000).isoformat(),
                "player_champion": player["championName"],
                "opponent_champion": opponent["championName"],
                "total_cs": player["totalMinionsKilled"],
                "opponent_total_cs": opponent["totalMinionsKilled"],
                "cs_per_min": player["totalMinionsKilled"] / game_duration,
                "gold_diff": player["goldEarned"] - opponent["goldEarned"],
                "first_10_min_cs": player["challenges"].get("laneMinionsFirst10Minutes", 0),
                "opponent_first_10_min_cs": opponent["challenges"].get("laneMinionsFirst10Minutes", 0),
                "win": player["win"],
                "player_cs_timeline": ",".join(
                    str(frame["participantFrames"][str(player["participantId"])]["minionsKilled"])
                    for frame in timeline_data["info"]["frames"][:15]
                ),
                "opponent_cs_timeline": ",".join(
                    str(frame["participantFrames"][str(opponent["participantId"])]["minionsKilled"])
                    for frame in timeline_data["info"]["frames"][:15]
                )
            }
            records.append(record)

        # Write to CSV
        if records:
            for record in records:
                writer.writerow(record)
            return True
        return False

    except Exception as e:
        logging.error(f"Failed to process match {match_id}: {str(e)}")
        return False

def main():
    """Main data collection loop"""
    # CSV file configuration
    csv_file = "matchup_data.csv"
    fieldnames = [
        "match_id", "timestamp", "player_champion", "opponent_champion",
        "total_cs", "opponent_total_cs", "cs_per_min", "gold_diff",
        "first_10_min_cs", "opponent_first_10_min_cs", "win",
        "player_cs_timeline", "opponent_cs_timeline"
    ]
    
    # Track processed matches
    processed_matches = set()
    if os.path.exists(csv_file):
        with open(csv_file, "r") as f:
            reader = csv.DictReader(f)
            processed_matches = {row["match_id"] for row in reader}
    
    # Handle interrupt signal
    def signal_handler(sig, frame):
        print("\nSaving data and exiting gracefully...")
        sys.exit(0)
    signal.signal(signal.SIGINT, signal_handler)

    # Main collection loop
    with open(csv_file, "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not processed_matches:
            writer.writeheader()

        while True:
            try:
                # Get fresh player list periodically
                players = get_players()
                logging.info(f"Processing {len(players)} players")

                for player in players:
                    # Get player identifier
                    puuid = get_puuid(player.get("summonerId"))
                    if not puuid:
                        continue

                    # Process player's matches
                    match_ids = get_match_history(puuid)
                    new_matches = [mid for mid in match_ids if mid not in processed_matches]
                    
                    for match_id in new_matches:
                        if process_match(match_id, writer):
                            processed_matches.add(match_id)
                            f.flush()  # Ensure immediate write
                            logging.info(f"Processed match {match_id[-8:]}")
                        else:
                            logging.warning(f"Skipped match {match_id[-8:]}")

                    time.sleep(0.1)  # Brief pause between players

                time.sleep(60)  # Wait before next batch

            except Exception as e:
                logging.error(f"Main loop error: {str(e)}")
                time.sleep(30)  # Wait before retrying

if __name__ == "__main__":
    main()

: 